# AlphaFold 3 Interactive Notebook

This notebook demonstrates how to use AlphaFold 3 for predicting complex biomolecular structures including:
- Protein-DNA interactions
- Protein-RNA interactions
- Protein-ligand binding
- Multi-component assemblies

**Important:** AlphaFold 3 is available through the [AlphaFold Server](https://alphafoldserver.com) (easiest for most users) and as [open-source code](https://github.com/google-deepmind/alphafold3) for local execution (requires Linux/Docker).

This notebook focuses on the **Server workflow**:
1. How to prepare inputs for AlphaFold 3
2. How to submit jobs via the web interface
3. How to visualize and analyze results
4. Example use cases for protein-DNA, protein-RNA, and protein-ligand complexes

## 1. Setup and Installation

Install required packages for working with AlphaFold 3 inputs and outputs.

In [ ]:
# Install required packages
%pip install -q biopython py3Dmol matplotlib requests rdkit

print("✓ All packages installed successfully!")

## 2. Import Required Libraries

In [ ]:
# Core libraries
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Visualization
import py3Dmol

# RDKit for ligand handling
try:
    from rdkit import Chem
    from rdkit.Chem import AllChem, Draw
    print("✓ RDKit available for ligand handling")
except ImportError:
    print("⚠ RDKit not available (optional, for ligand visualization)")

print("✓ All libraries imported successfully!")

## 3. AlphaFold 3 Capabilities Overview

AlphaFold 3 extends beyond proteins to predict:
- **Proteins** - Enhanced accuracy over AlphaFold 2
- **DNA/RNA** - Nucleic acid structures and protein-nucleic acid interactions
- **Ligands** - Small molecule binding (drugs, metabolites, cofactors)
- **Ions** - Metal coordination (Mg²⁺, Ca²⁺, Zn²⁺, Fe²⁺/³⁺)
- **Modifications** - Post-translational modifications
- **Complexes** - Multi-component biomolecular assemblies

In [ ]:
# Display comparison table
import pandas as pd

capabilities = {
    "Feature": ["Protein structure", "Protein complexes", "DNA/RNA", "Ligands", "Ions", "PTMs", "Accuracy"],
    "AlphaFold 2": ["✓", "✓", "✗", "✗", "✗", "✗", "High"],
    "AlphaFold 3": ["✓✓", "✓✓", "✓", "✓", "✓", "✓", "Higher"],
}

df = pd.DataFrame(capabilities)
print("AlphaFold 2 vs AlphaFold 3 Capabilities:")
print(df.to_string(index=False))

## 4. Example 1: Protein-DNA Complex (Transcription Factor)

**Use Case:** Understanding how proteins recognize specific DNA sequences

This example shows a zinc finger protein binding to DNA promoter sequence.

In [ ]:
# Define protein-DNA complex
protein_dna_complex = {
    "job_name": "zinc_finger_transcription_factor",
    "components": [
        {
            "type": "protein",
            "name": "ZincFingerProtein",
            "sequence": "MKLKFSLLTHVKLPVPGDKVEVRCPHYTCPVCGKSFSQKSDLVKHQRTHTG",
            "description": "Zinc finger DNA-binding domain"
        },
        {
            "type": "dna",
            "name": "PromoterDNA",
            "sequence": "ATGCTAGCTAGCTAGCTA",  # DNA is automatically paired
            "description": "DNA promoter region (18 bp)"
        }
    ]
}

# Display input
print("Protein-DNA Complex Input:")
print("=" * 60)
for component in protein_dna_complex["components"]:
    print(f"\n{component['type'].upper()}: {component['name']}")
    print(f"  Sequence: {component['sequence']}")
    print(f"  Length: {len(component['sequence'])} {'residues' if component['type']=='protein' else 'base pairs'}")
    print(f"  Description: {component['description']}")

print("\n" + "=" * 60)
print("\nExpected Output:")
print("  - 3D structure showing zinc finger motifs inserted into DNA major groove")
print("  - Confidence scores for protein-DNA contacts")
print("  - Interface contact probabilities")

# Save input as JSON (for AlphaFold Server API submission)
output_dir = Path("./af3_inputs")
output_dir.mkdir(exist_ok=True)

input_json = output_dir / "protein_dna_complex.json"
with open(input_json, 'w') as f:
    json.dump(protein_dna_complex, f, indent=2)

print(f"\n✓ Input saved to: {input_json}")

## 5. Example 2: Protein-RNA Complex (CRISPR)

**Use Case:** Gene editing complex structure prediction

This example shows Cas9 protein bound to guide RNA.

In [ ]:
# Define protein-RNA complex
# Note: Using shortened Cas9 sequence for demo
protein_rna_complex = {
    "job_name": "crispr_cas9_guide_rna",
    "components": [
        {
            "type": "protein",
            "name": "Cas9_Protein",
            "sequence": "MDKKYSIGLDIGTNSVGWAVITDEYKVPSKKFKVLGNTDRHSIKKNLIGALLFDSGETAEATRLKRTARRRYTRRKNRICYLQEIFSNEMAKVDDSFFHRLEESFLVEEDKKHERHPIFGNIVDEVAYHEKYPTIYHLRKKLVDSTDKADLRLIYLALAHMIKFRGHFLIEGDLNPDNSDVDKLFIQLVQTYNQLFEENPINASGVDAKAILSARLSKSRRLENLIAQLPGEKKNGLFGNLIALSLGLTPNFKSNFDLAEDAKLQLSKDTYDDDLDNLLAQIGDQYADLFLAAKNLSDAILLSDILRVNTEITKAPLSASMIKRYDEHHQDLTLLKALVRQQLPEKYKEIFFDQSKNGYAGYIDGGASQEEFYKFIKPILEKMDGTEELLVKLNREDLLRKQRTFDNGSIPHQIHLGELHAILRRQEDFYPFLKDNREKIEKILTFRIPYYVGPLARGNSRFAWMTRKSEETITPWNFEEVVDKGASAQSFIERMTNFDKNLPNEKVLPKHSLLYEYFTVYNELTKVKYVTEGMRKPAFLSGEQKKAIVDLLFKTNRKVTVKQLKEDYFKKIECFDSVEISGVEDRFNASLGTYHDLLKIIKDKDFLDNEENEDILEDIVLTLTLFEDREMIEERLKTYAHLFDDKVMKQLKRRRYTGWGRLSRKLINGIRDKQSGKTILDFLKSDGFANRNFMQLIHDDSLTFKEDIQKAQVSGQGDSLHEHIANLAGSPAIKKGILQTVKVVDELVKVMGRHKPENIVIEMARENQTTQKGQKNSRERMKRIEEGIKELGSQILKEHPVENTQLQNEKLYLYYLQNGRDMYVDQELDINRLSDYDVDHIVPQSFLKDDSIDNKVLTRSDKNRGKSDNVPSEEVVKKMKNYWRQLLNAKLITQRKFDNLTKAERGGLSELDKAGFIKRQLVETRQITKHVAQILDSRMNTKYDENDKLIREVKVITLKSKLVSDFRKDFQFYKVREINNYHHAHDAYLNAVVGTALIKKYPKLESEFVYGDYKVYDVRKMIAKSEQEIGKATAKYFFYSNIMNFFKTEITLANGEIRKRPLIETNGETGEIVWDKGRDFATVRKVLSMPQVNIVKKTEVQTGGFSKESILPKRNSDKLIARKKDWDPKKYGGFDSPTVAYSVLVVAKVEKGKSKKLKSVKELLGITIMERSSFEKNPIDFLEAKGYKEVKKDLIIKLPKYSLFELENGRKRMLASAGELQKGNELALPSKYVNFLYLASHYEKLKGSPEDNEQKQLFVEQHKHYLDEIIEQISEFSKRVILADANLDKVLSAYNKHRDKPIREQAENIIHLFTLTNLGAPAAFKYFDTTIDRKRYTSTKEVLDATLIHQSITGLYETRIDLSQLGGD",
            "description": "Cas9 endonuclease (shortened sequence for demo)"
        },
        {
            "type": "rna",
            "name": "GuideRNA",
            "sequence": "GUUUUAGAGCUAGAAAUAGCAAGUUAAAAUAAGGC",
            "description": "Single guide RNA (sgRNA)"
        }
    ]
}

print("Protein-RNA Complex Input:")
print("=" * 60)
for component in protein_rna_complex["components"]:
    print(f"\n{component['type'].upper()}: {component['name']}")
    if component['type'] == 'protein':
        print(f"  Sequence: {component['sequence'][:50]}... (truncated)")
        print(f"  Full length: {len(component['sequence'])} residues")
    else:
        print(f"  Sequence: {component['sequence']}")
        print(f"  Length: {len(component['sequence'])} nucleotides")
    print(f"  Description: {component['description']}")

print("\n" + "=" * 60)
print("\nExpected Output:")
print("  - Cas9 structure with RNA guide in the active site channel")
print("  - RNA binding groove and target DNA recognition site")
print("  - High confidence for RNA-protein interface")

# Save input
input_json = output_dir / "protein_rna_complex.json"
with open(input_json, 'w') as f:
    json.dump(protein_rna_complex, f, indent=2)

print(f"\n✓ Input saved to: {input_json}")

## 6. Example 3: Protein-Ligand Complex (Drug Binding)

**Use Case:** Drug discovery and binding site analysis

This example shows a protein kinase with an ATP-competitive inhibitor.

In [ ]:
# Define protein-ligand complex
protein_ligand_complex = {
    "job_name": "kinase_inhibitor_complex",
    "components": [
        {
            "type": "protein",
            "name": "Protein_Kinase",
            "sequence": "MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELRHPNIVSLQDVLMQDSRLYLIFEFLSMDLKKYLDSSIPPKD",
            "description": "Protein kinase catalytic domain"
        },
        {
            "type": "ligand",
            "name": "Kinase_Inhibitor",
            "smiles": "CC1=C2C(=NC=C1)N(C3=C(C=CC=C3)C2=O)CC4=CC=CC=C4",
            "description": "ATP-competitive kinase inhibitor"
        }
    ]
}

print("Protein-Ligand Complex Input:")
print("=" * 60)
for component in protein_ligand_complex["components"]:
    print(f"\n{component['type'].upper()}: {component['name']}")
    if component['type'] == 'protein':
        print(f"  Sequence: {component['sequence']}")
        print(f"  Length: {len(component['sequence'])} residues")
    else:
        print(f"  SMILES: {component['smiles']}")
    print(f"  Description: {component['description']}")

print("\n" + "=" * 60)
print("\nExpected Output:")
print("  - Inhibitor bound in ATP binding pocket")
print("  - Key hydrogen bonds and hydrophobic interactions")
print("  - Ligand binding confidence score")

# Visualize ligand structure (if RDKit available)
try:
    mol = Chem.MolFromSmiles(protein_ligand_complex["components"][1]["smiles"])
    if mol:
        # Generate 2D coordinates
        AllChem.Compute2DCoords(mol)
        # Display molecular structure
        img = Draw.MolToImage(mol, size=(400, 400))
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title("Kinase Inhibitor Structure (2D)", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Print molecular properties
        print("\nLigand Properties:")
        print(f"  Molecular Formula: {Chem.rdMolDescriptors.CalcMolFormula(mol)}")
        print(f"  Molecular Weight: {Chem.rdMolDescriptors.CalcExactMolWt(mol):.2f} g/mol")
        print(f"  Number of Atoms: {mol.GetNumAtoms()}")
        print(f"  Number of Bonds: {mol.GetNumBonds()}")
except Exception as e:
    print(f"\n⚠ RDKit not available for ligand visualization: {str(e)}")

# Save input
input_json = output_dir / "protein_ligand_complex.json"
with open(input_json, 'w') as f:
    json.dump(protein_ligand_complex, f, indent=2)

print(f"\n✓ Input saved to: {input_json}")

## 7. How to Submit to AlphaFold Server

**AlphaFold 3 is available via the AlphaFold Server web interface:**

### Step-by-Step Workflow:

1. **Access**: Go to [https://alphafoldserver.com](https://alphafoldserver.com)
2. **Sign in**: Use Google account (free for non-commercial use, with usage limits)
3. **Create job**: Click "New Prediction" or "Create Job"
4. **Add components**:
   - Paste protein sequence or upload FASTA file
   - Add DNA/RNA sequences (A,T,G,C for DNA or A,U,G,C for RNA)
   - Add ligands using SMILES notation or PubChem CID
   - Add ions from dropdown (Mg²⁺, Ca²⁺, Zn²⁺, Fe²⁺, etc.)
   - Add post-translational modifications if needed
5. **Configure**: Set job name and description
6. **Submit**: Run prediction (typically takes 10-30 minutes depending on complexity)
7. **Download**: Get PDB file + confidence metrics (JSON) + visualizations

### Usage Limits:

- Free tier: Limited number of predictions per day
- Academic use: More generous limits for research
- For high-throughput needs, consider local installation (requires Linux/Docker and model weights).

### Note on API:

Currently, the AlphaFold Server operates through the web interface. For programmatic access, monitor the [official documentation](https://github.com/google-deepmind/alphafold3) for updates.

In [ ]:
# Demonstration: Placeholder for future API submission
# This code shows what API submission might look like

def submit_to_alphafold_server(job_config):
    """
    Placeholder function for AlphaFold Server API submission.
    
    When API becomes available, this would:
    1. Authenticate with API key
    2. Submit job configuration
    3. Poll for completion
    4. Download results
    """
    print("⚠ Note: AlphaFold Server currently uses web interface only")
    print("\nJob would be submitted with:")
    print(f"  Job name: {job_config['job_name']}")
    print(f"  Components: {len(job_config['components'])}")
    for comp in job_config['components']:
        print(f"    - {comp['type']}: {comp['name']}")
    
    print("\nPlease use the web interface at:")
    print("  https://alphafoldserver.com")
    print("\nFor programmatic generation, you can now run AlphaFold 3 locally using the open-source release:")
    print("  https://github.com/google-deepmind/alphafold3")
    
    return None

# Example usage

print("Example: Submitting protein-DNA complex\n")
submit_to_alphafold_server(protein_dna_complex)

## 8. Analyzing AlphaFold 3 Results

Once you download results from AlphaFold Server, you'll get:
- **PDB file**: 3D structure of the complex
- **Confidence metrics**: pLDDT, PAE, pTM, ipTM
- **Contact probabilities**: For ligands and interfaces

Let's demonstrate how to analyze these results (using a sample structure).

In [ ]:
# Function to visualize AlphaFold 3 results
def visualize_af3_structure(pdb_file_path, show_ligands=True):
    """
    Visualize AlphaFold 3 predicted structure with confidence coloring.
    
    Args:
        pdb_file_path: Path to PDB file from AlphaFold Server
        show_ligands: Whether to highlight ligands/nucleic acids
    """
    with open(pdb_file_path, 'r') as f:
        pdb_data = f.read()
    
    # Create 3D viewer
    view = py3Dmol.view(width=800, height=600)
    view.addModel(pdb_data, 'pdb')
    
    # Color protein by confidence (pLDDT in B-factor)
    view.setStyle({'chain': 'A'}, {'cartoon': {'colorscheme': 'ssBfactor'}})
    
    if show_ligands:
        # Highlight DNA/RNA as sticks
        view.setStyle({'hetflag': True}, {'stick': {'colorscheme': 'greenCarbon'}})
        
    view.zoomTo()
    return view

# Example: Create a sample PDB for demonstration
sample_pdb = """ATOM      1  N   MET A   1      10.000  10.000  10.000  1.00 95.00           N
ATOM      2  CA  MET A   1      11.000  10.000  10.000  1.00 95.00           C
ATOM      3  C   MET A   1      11.500  11.000  10.000  1.00 95.00           C
ATOM      4  O   MET A   1      11.000  12.000  10.000  1.00 95.00           O
"""

print("Sample structure analysis workflow:")
print("=" * 60)
print("\n1. Load PDB file from AlphaFold Server download")
print("2. Extract confidence metrics (pLDDT, PAE, ipTM)")
print("3. Identify interface residues (protein-DNA, protein-ligand)")
print("4. Calculate contact probabilities")
print("5. Visualize structure with confidence coloring")
print("\n" + "=" * 60)

print("\n✓ When you have actual AlphaFold 3 results, use:")
print("  view = visualize_af3_structure('path/to/your/structure.pdb')")
print("  view.show()")

## 9. AlphaFold 3 Confidence Metrics

AlphaFold 3 provides enhanced confidence metrics beyond AlphaFold 2:

### Metrics:
- **pLDDT** (0-100): Per-residue confidence (same as AF2)
  - >90 = Very confident
  - 70-90 = Confident
  - <70 = Low confidence
  
- **PAE** (Å): Predicted aligned error between residue pairs
  - Lower is better
  - Blue regions = confident positioning
  
- **pTM** (0-1): Overall structure confidence
  - >0.5 = Good fold
  
- **ipTM** (0-1): Interface confidence
  - >0.5 = Confident interface
  - <0.3 = Uncertain complex orientation
  
- **Contact probability**: For ligand binding
  - Higher = More confident binding pose

In [ ]:
# Visualize confidence metric thresholds
fig, ax = plt.subplots(figsize=(12, 6))

metrics = ['pLDDT\n(per-residue)', 'pTM\n(structure)', 'ipTM\n(interface)', 'PAE\n(Ångströms)']
very_good = [90, 0.8, 0.7, 5]
good = [70, 0.5, 0.5, 15]
uncertain = [50, 0.3, 0.3, 25]

x = np.arange(len(metrics))
width = 0.25

bars1 = ax.bar(x - width, very_good, width, label='Very Confident', color='#2ecc71', alpha=0.8)
bars2 = ax.bar(x, good, width, label='Confident', color='#f39c12', alpha=0.8)
bars3 = ax.bar(x + width, uncertain, width, label='Low Confidence', color='#e74c3c', alpha=0.8)

ax.set_ylabel('Score / Value', fontsize=12)
ax.set_title('AlphaFold 3 Confidence Metric Thresholds', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend(fontsize=10)
ax.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\n✅ Interpretation Guidelines:")
print("\n1. Protein regions: Focus on pLDDT > 70")
print("2. Overall fold: Check pTM > 0.5")
print("3. Interfaces: Look for ipTM > 0.5")
print("4. Ligand binding: High contact probability + low PAE at interface")
print("5. Compare with experimental data when available")

## 🎉 Summary and Next Steps

### What You've Learned:
✓ AlphaFold 3's extended capabilities (DNA/RNA, ligands, ions)  
✓ How to prepare inputs for different complex types  
✓ Submitting jobs to AlphaFold Server  
✓ Understanding confidence metrics  
✓ Visualizing and analyzing results  

### Your Input Files Are Ready!
All example inputs have been saved to the `af3_inputs/` directory. You can use these as templates for your own predictions.

### Next Steps:
1. **Submit your first job**: Go to [AlphaFold Server](https://alphafoldserver.com)
2. **Upload sequences**: Use the JSON files we created as reference (manually input sequences in the web interface)
3. **Download results**: Get PDB files and confidence metrics
4. **Analyze**: Use the visualization functions in this notebook
5. **Compare**: Validate with experimental structures when available

### Use Cases to Try:
- **Transcription factors**: Predict DNA-binding specificity
- **CRISPR systems**: Understand guide RNA recognition
- **Drug discovery**: Predict ligand binding poses
- **Enzyme-substrate**: Model catalytic mechanisms
- **Antibody-antigen**: Design therapeutic antibodies

### Resources:
- **AlphaFold 3 Paper**: [Nature (2024)](https://www.nature.com/articles/s41586-024-07487-w)
- **AlphaFold Server**: [alphafoldserver.com](https://alphafoldserver.com)
- **Official Documentation**: [github.com/google-deepmind/alphafold3](https://github.com/google-deepmind/alphafold3)
- **AlphaFold Database**: [alphafold.ebi.ac.uk](https://alphafold.ebi.ac.uk/)
- **Community & Updates**: [BioRxiv preprints](https://www.biorxiv.org/) and [@GoogleDeepMind](https://twitter.com/GoogleDeepMind)